## 営業成約予測（分類） Model

In [1]:
# 共通事前処理

# 余分な Warning を非常にする
import warnings

warnings.filterwarnings('ignore')

# Library の import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib 日本語化対応
import japanize_matplotlib

# DataFrame 表示用関数
from IPython.display import display

# 表示 Option の調整
# NumPy の浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)
# pandas での浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format
# DataFrame ですべての項目を表示
pd.set_option('display.max_columns', None)
# Graph の Default Font 指定
plt.rcParams['font.size'] = 14
# 乱数の種
random_seed = 123

### 公開 DataSet を DataFrame に読み込み

In [2]:
# 公開 Data の Download と解凍
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip -O bank.zip | tail -n 1
# !unzip -o bank.zip | tail -n 1

In [3]:
# bank-full.csv を DataFrame に読み込み
df_all = pd.read_csv('bank-full.csv', sep=';') # 項目の区切り文字が Comma ではなく SemiCoron になっている為 `sep=';'` で区切り文字を指定。

In [4]:
# 項目名を日本語に置き換える
columns = ['年齢', '職業', '婚姻', '学歴', '債務不履行', '平均残高', '住宅_Loan', '個人_Loan', '連絡手段', '最終通話日', '最終通話月', '最終通話秒数', '通話回数_販促中', '前回販促後_経過日数', '通話回数_販促前', '前回販促結果', '今回販促結果']

df_all.columns = columns

## DataFrame の内容確認

In [5]:
display(df_all.head())

,年齢,職業,婚姻,学歴,債務不履行,平均残高,住宅_Loan,個人_Loan,連絡手段,最終通話日,最終通話月,最終通話秒数,通話回数_販促中,前回販促後_経過日数,通話回数_販促前,前回販促結果,今回販促結果
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


かなりの項目値が Text Data になっている。
機械学習 Model に入力できるような項目値が数値だけからなる Data に変換していく。

## Data の確認

In [11]:
# 学習 Data の件数と項目数確認
print(df_all.shape)
print()

# 「今回販促結果」の値の分布確認
print(df_all['今回販促結果'].value_counts())
print()

# 営業成功率
rate = df_all['今回販促結果'].value_counts()['yes']/len(df_all)
print(f'営業成功率: {rate:.4f}')

(45211, 17)

no     39922
yes     5289
Name: 今回販促結果, dtype: int64

営業成功率: 0.1170


## 欠損値の確認

In [13]:
# 欠損値の確認
print(df_all.isnull().sum())

年齢            0
職業            0
婚姻            0
学歴            0
債務不履行         0
平均残高          0
住宅_Loan       0
個人_Loan       0
連絡手段          0
最終通話日         0
最終通話月         0
最終通話秒数        0
通話回数_販促中      0
前回販促後_経過日数    0
通話回数_販促前      0
前回販促結果        0
今回販促結果        0
dtype: int64


欠損値は一切なく、対応不要。